In [18]:
import trimesh

from fidget.types import GearConfig, ShapeConfig, ShapeType, PinConfig, PrintLayoutConfig
from fidget.ops import build_core, build_gear, align_gear, rotate_gear, build_shape, apply_shape, build_pin, add_holes, add_hole, align_gears_vertically, arrange_meshes


rotation_angles = [27, 17, 7, -3, -3, 7, 17, 27]
CORE_DIAMETER = 35.0

core = build_core(diameter=CORE_DIAMETER)
gear_config = GearConfig(profile="./assets/gear.svg", base_diameter=24.0, height=500.0, angle=5.7)

def process_gear(gear_config, face, angle):
    gear = build_gear(gear_config)
    aligned_gear = align_gear(gear, face, 0)
    return rotate_gear(aligned_gear, face, angle)

gears = [process_gear(gear_config, face, angle) 
         for angle, face in zip(rotation_angles, core.faces)]


# shape_config = ShapeConfig(
#     shape_type=ShapeType.CUBE, 
#     size=[40.0, 40.0, 40.0], 
#     scale=(1.0, 1.0, 1.0), 
#     rotation=(0, 0, 0), 
#     offset=(0, 0, 0)
# )

shape_config = ShapeConfig(
    shape_type=ShapeType.CUSTOM, 
    stl_path="./assets/hyperboloid.stl", 
    rotation=(0, 0, 0), 
    offset=(0, 0, 0), 
    scale=(2.5, 2.5, 2.5)
)

shape = build_shape(shape_config)

cut_gears = apply_shape(shape, gears)

scene = trimesh.Scene([core.mesh, *[rotate_gear(gear, face, 0) for gear, face in zip(cut_gears, core.faces)]])
scene.show()

pin = build_pin(PinConfig(diameter=6.0, length=12.0, center_length=2.0))
core_with_holes = add_holes(core, pin, 1.05)

cut_gears_with_holes = [add_hole(gear, face, pin, 1.05) for gear, face in zip(cut_gears, core.faces)]

scene = trimesh.Scene([core_with_holes, *cut_gears_with_holes])
scene.show()

In [3]:
arranged_gears = arrange_meshes(align_gears_vertically(cut_gears_with_holes, core.faces), PrintLayoutConfig(row_spacing=35, col_spacing=35, rows=3, cols=3))
pins = arrange_meshes([pin.pin_mesh.copy() for _ in range(8)], PrintLayoutConfig(row_spacing=23, col_spacing=10, rows=3, cols=3))

def save_stl():
    trimesh.Scene(pins).export("pins.stl")
    for i, gear in enumerate(arranged_gears):
        gear.export(f"gear_{i}.stl")
    core_with_holes.mesh.export("core.stl")

save_stl()